In [1]:
# Dependencies and config file
import csv
import matplotlib.pyplot as plt
import requests
import pandas as pd
from config import api_key, api_key2, api_key3, api_key4, api_key5, api_key6, api_key7
import openweathermapy as owm
from citipy import citipy
from random import randint
import numpy as np

In [2]:
# Randomly select 500 unique latitudes
latitude_list = []

for x in range(2500):
    latitude_list.append(np.random.uniform(low=-180.0, high=180, size=None))

# Convert list to series
latitude_list_series = pd.Series(latitude_list)

In [3]:
# Randomly select 500 unique longitudes
longitude_list = []

for x in range(2500):
    longitude_list.append(np.random.uniform(low=-90, high = 90))
    #print(longitude)

# Convert list to series
longitude_list_series = pd.Series(longitude_list)

In [4]:
# Concatentate the series into a df
lat_and_long = pd.concat([latitude_list_series, longitude_list_series], axis=1)
#lat_and_long.head(10)
#lat_and_long = lat_and_long.set_index(0)
lat_and_long.head(10)

,0,1
0,-168.981339,33.205935
1,-125.539019,73.353960
2,-53.594493,27.620464
3,-126.322937,31.695027
4,97.462264,-42.403755
5,-160.736272,55.766976
6,-113.315763,2.977924
7,-61.359799,-50.615602
8,143.247641,-60.051966
9,-56.961349,-28.793997


In [5]:
# Create new empty columns and add to existing dataframe 
lat_and_long["City"] = ("")
lat_and_long["Country Code"] = ("")
lat_and_long["Temperature"] =("")
lat_and_long["Cloudiness"] =("")
lat_and_long["Windspeed"] =("")
lat_and_long.head(10)
# Rename columns
lat_and_long = lat_and_long.rename(columns={0: "Latitude", 1:"Longitude"})
lat_and_long.head()

,Latitude,Longitude,City,Country Code,Temperature,Cloudiness,Windspeed
0,-168.981339,33.205935,,,,,
1,-125.539019,73.353960,,,,,
2,-53.594493,27.620464,,,,,
3,-126.322937,31.695027,,,,,
4,97.462264,-42.403755,,,,,


In [6]:
# Blank column for holiding the city name
cities = []

#row_counter = 0
for index, row in lat_and_long.iterrows():
    x = row["Latitude"]
    y = row["Longitude"]
    city = citipy.nearest_city(x,y)
    lat_and_long.set_value(index, "City", citipy.nearest_city(x, y).city_name)
    lat_and_long.set_value(index, "Country Code", citipy.nearest_city(x, y).country_code)
    city_list = city.city_name
    cities.append(city_list)
#lat_and_long
#print(len(city_list))
print(cities)

#cities = pd.Series(cities)

['bredasdorp', 'albany', 'port elizabeth', 'bredasdorp', 'upernavik', 'port elizabeth', 'hermanus', 'ushuaia', 'qaanaaq', 'chuy', 'vila franca do campo', 'narsaq', 'vestmannaeyjar', 'taolanaro', 'port elizabeth', 'loyga', 'toulepleu', 'barbar', 'bredasdorp', 'ushuaia', 'cape town', 'hermanus', 'dikson', 'port alfred', 'taolanaro', 'hermanus', 'bredasdorp', 'longyearbyen', 'longyearbyen', 'east london', 'lasa', 'albany', 'coquimbo', 'kutum', 'ushuaia', 'nanortalik', 'qaanaaq', 'barentsburg', 'qaanaaq', 'hermanus', 'barentsburg', 'albany', 'qaanaaq', 'longyearbyen', 'bredasdorp', 'belushya guba', 'longyearbyen', 'amderma', 'ponta do sol', 'vila velha', 'ushuaia', 'port elizabeth', 'ponta do sol', 'belushya guba', 'port alfred', 'port alfred', 'barentsburg', 'grand river south east', 'hermanus', 'taoudenni', 'tsihombe', 'ushuaia', 'upernavik', 'ilulissat', 'hermanus', 'pisco', 'longyearbyen', 'qaanaaq', 'illoqqortoormiut', 'qaanaaq', 'illoqqortoormiut', 'belushya guba', 'albany', 'hermanu

In [7]:
lat_and_long.head()

,Latitude,Longitude,City,Country Code,Temperature,Cloudiness,Windspeed
0,-168.981339,33.205935,bredasdorp,za,,,
1,-125.539019,73.353960,albany,au,,,
2,-53.594493,27.620464,port elizabeth,za,,,
3,-126.322937,31.695027,bredasdorp,za,,,
4,97.462264,-42.403755,upernavik,gl,,,


In [8]:
lat_and_long = lat_and_long.drop_duplicates(subset = ["City"], keep = "first")
lat_and_long = lat_and_long.reset_index(drop=True)
lat_and_long.head()

,Latitude,Longitude,City,Country Code,Temperature,Cloudiness,Windspeed
0,-168.981339,33.205935,bredasdorp,za,,,
1,-125.539019,73.353960,albany,au,,,
2,-53.594493,27.620464,port elizabeth,za,,,
3,97.462264,-42.403755,upernavik,gl,,,
4,-113.315763,2.977924,hermanus,za,,,


In [9]:
# Create empty list for storing city + country code together
city_country_list = []

# Iterate through df to populate the list of cities with their country code
for index, row in lat_and_long.iterrows():
    x = row["City"]
    y = row["Country Code"]
    city = str(x) + "," + str(y)
    #print(city)
    if city not in city_country_list:
        city_country_list.append(city) 
#print(city_country_list)

In [25]:
# Create settings dictionary and summary list
settings = {"units": "metric", "appid": api_key3}
#summary = ["name","main.temp", "wind.speed", "clouds.all"]

meta_weather = {}
# loop through the city,country list to get the current weather
for x in city_country_list:
    print(f"Retrieving data for city:{x}.")
    try:
        current_weather = owm.get_current(x, **settings)
        temp = current_weather("main.temp")
        wind_speed = current_weather("wind.speed")
        print(f"the temp is:{temp}.")
        print(f"the wind speed is:{wind.speed}.")
    except: 
        print("No data found.")


Retrieving data for city:bredasdorp,za.
the temp is:30.
No data found.
Retrieving data for city:albany,au.
the temp is:16.
No data found.
Retrieving data for city:port elizabeth,za.
the temp is:25.
No data found.
Retrieving data for city:upernavik,gl.
the temp is:-23.98.
No data found.
Retrieving data for city:hermanus,za.
the temp is:27.68.
No data found.
Retrieving data for city:ushuaia,ar.
the temp is:4.12.
No data found.
Retrieving data for city:qaanaaq,gl.
the temp is:-27.35.
No data found.
Retrieving data for city:chuy,uy.
the temp is:20.05.
No data found.
Retrieving data for city:vila franca do campo,pt.
the temp is:17.
No data found.
Retrieving data for city:narsaq,gl.
the temp is:-9.
No data found.
Retrieving data for city:vestmannaeyjar,is.
the temp is:5.
No data found.
Retrieving data for city:taolanaro,mg.
No data found.
Retrieving data for city:loyga,ru.
the temp is:-6.88.
No data found.
Retrieving data for city:toulepleu,ci.
No data found.
Retrieving data for city:barbar,

the temp is:-14.38.
No data found.
Retrieving data for city:cuamba,mz.
the temp is:30.05.
No data found.
Retrieving data for city:amapa,br.
the temp is:26.1.
No data found.
Retrieving data for city:mahebourg,mu.
the temp is:28.
No data found.
Retrieving data for city:kuche,cn.
No data found.
Retrieving data for city:caravelas,br.
the temp is:27.38.
No data found.
Retrieving data for city:tazovskiy,ru.
the temp is:-16.75.
No data found.
Retrieving data for city:bay city,us.
the temp is:5.18.
No data found.
Retrieving data for city:rawson,ar.
the temp is:14.35.
No data found.
Retrieving data for city:arraial do cabo,br.
the temp is:26.97.
No data found.
Retrieving data for city:iskateley,ru.
the temp is:-13.7.
No data found.
Retrieving data for city:sergeyevka,kz.
the temp is:-0.65.
No data found.
Retrieving data for city:gimbi,et.
the temp is:25.3.
No data found.
Retrieving data for city:katiola,ci.
the temp is:28.
No data found.
Retrieving data for city:santa rosa,ar.
the temp is:12.03

the temp is:25.
No data found.
Retrieving data for city:kudahuvadhoo,mv.
the temp is:28.78.
No data found.
Retrieving data for city:pascagoula,us.
the temp is:15.5.
No data found.
Retrieving data for city:ovria,gr.
the temp is:16.
No data found.
Retrieving data for city:stromness,gb.
the temp is:5.
No data found.
Retrieving data for city:talnakh,ru.
the temp is:-8.15.
No data found.
Retrieving data for city:novaci,mk.
the temp is:10.
No data found.
Retrieving data for city:usinsk,ru.
the temp is:-16.23.
No data found.
Retrieving data for city:bay roberts,ca.
the temp is:-2.47.
No data found.
Retrieving data for city:pedernales,do.
the temp is:23.
No data found.
Retrieving data for city:coihaique,cl.
the temp is:10.
No data found.
Retrieving data for city:zavyalovo,ru.
the temp is:-1.
No data found.
Retrieving data for city:krutikha,ru.
the temp is:-2.73.
No data found.
Retrieving data for city:bajil,ye.
the temp is:33.18.
No data found.
Retrieving data for city:kamenka,ru.
the temp is:

the temp is:31.
No data found.
Retrieving data for city:bousso,td.
the temp is:41.8.
No data found.
Retrieving data for city:grand gaube,mu.
the temp is:28.
No data found.
Retrieving data for city:carauari,br.
the temp is:23.38.
No data found.
Retrieving data for city:shebalino,ru.
the temp is:0.5.
No data found.
Retrieving data for city:cap malheureux,mu.
the temp is:28.
No data found.
Retrieving data for city:port shepstone,za.
the temp is:26.8.
No data found.
Retrieving data for city:horta,pt.
the temp is:15.
No data found.
Retrieving data for city:awjilah,ly.
the temp is:32.1.
No data found.
Retrieving data for city:samarkand,uz.
the temp is:20.
No data found.
Retrieving data for city:beckley,us.
the temp is:1.93.
No data found.
Retrieving data for city:goderich,sl.
No data found.
Retrieving data for city:hofn,is.
the temp is:3.4.
No data found.
Retrieving data for city:lernakert,am.
the temp is:16.37.
No data found.
Retrieving data for city:mandera,ke.
the temp is:38.18.
No data f

Retrieving data for city:marawi,sd.
the temp is:38.23.
No data found.
Retrieving data for city:blacksburg,us.
the temp is:0.5.
No data found.
Retrieving data for city:sorvag,fo.
No data found.
Retrieving data for city:canutama,br.
the temp is:25.43.
No data found.
Retrieving data for city:barkly west,za.
the temp is:29.
No data found.
Retrieving data for city:keetmanshoop,na.
the temp is:35.
No data found.
Retrieving data for city:chittaurgarh,in.
the temp is:35.35.
No data found.
Retrieving data for city:maloshuyka,ru.
No data found.
Retrieving data for city:kolkhikon,gr.
No data found.
Retrieving data for city:latur,in.
the temp is:36.15.
No data found.
Retrieving data for city:bara,sd.
No data found.
Retrieving data for city:buchanan,lr.
the temp is:30.
No data found.
Retrieving data for city:oranjestad,an.
No data found.
Retrieving data for city:atar,mr.
the temp is:30.8.
No data found.
Retrieving data for city:santa maria,cv.
the temp is:22.44.
No data found.
Retrieving data for c

In [22]:
#summary = ["main.temp", "wind.speed", "clouds.all"]
settings = {"units": "metric", "appid": api_key2}
current_weather_test = owm.get_current('albany,au', **settings)
#print(current_weather_test)
#for x in current_weather:
temp = current_weather_test("main.temp")
print(temp)
    #print(temp)
#data = current_weather(*summary)
#print(data)

16


In [ ]:
# Include a print log of each city as it is being processed with the city number, city name, and requested URL.
#print(f"Current weather object for {cities}: {data}.") 

In [ ]:
# Save a CSV of all data retrieved

In [ ]:
# Your objective is to build a series of scatter plots to showcase the following relationships:

# Temperature (F) vs. Latitude
# Humidity (%) vs. Latitude
# Cloudiness (%) vs. Latitude
# Wind Speed (mph) vs. Latitude

In [ ]:
# Build a scatter plot for each data type

#plt.scatter(df["temperature"].astype(int),
            #df["latitude"],
            #edgecolor="black", linewidths=1, marker="o",
            #alpha=0.8, label="?")

# Label the graph and set axis limits
#plt.title("")
#plt.ylabel("")
#plt.xlabel("")
#plt.grid(True)
#plt.xlim([, ])
#plt.ylim([, ])

# Save the figure as png image (for each scatter plot) 
#plt.savefig("UnemploymentRate_BankCount.png")

# Show plot
#plt.show()